In [13]:
from helpers import read_data, get_files, read_pcd, make_homogeneous, clean_pcds
from ICP import icp
import os
from merge_scenes import get_merge_transformations, merge_frames
from functools import reduce
from scipy.io import savemat
import itertools
import time
import matlab.engine
import numpy as np

np.random.seed(42)

In [6]:
# start matlab engine to preprocess files and visualize scatter plots
matlab_eng = matlab.engine.start_matlab()

In [3]:
data_path = '../data/'
transformed_data_path = '../data/transformed_data'

In [4]:
data_dir = os.path.join(os.getcwd(), f'{data_path}')
transformed_data_dir = os.path.join(os.getcwd(), f'{transformed_data_path}/')

In [7]:
# convert *.mat files to *.txt and regenerate normals
matlab_eng.generate_files(nargout=0)
matlab_eng.generate_normals(nargout=0)

the MATLAB function has been cancelled


MatlabExecutionError: 
  File /Users/andrewsklyar/Education/uva/computer-vision/part2/assignments/computer-vision-2/Assignment_1/final_code/generate_normals.m, line 6, in generate_normals
Unable to read file '/Users/andrewsklyar/Education/uva/computer-vision/part2/assignments/computer-vision-2/Assignment_1/data/data_mat/0000000001.mat'. No such file or directory.


In [5]:
# Run ICP for dummy files
source = read_data(os.path.join(data_dir, 'source.txt'))
target = read_data(os.path.join(data_dir, 'target.txt'))
max_iters = 200
T_cumul, full_X, mean_error, time_taken = icp(source.T, target.T, tolerance=1e-5)
savemat(os.path.join(data_dir, 'pcd_dummy.mat'), {'points': full_X})

Initial MSE: 0.331561847750409
MSE after iteration 0: 0.053988832066168176, difference between errors: 0.053988832066168176, tolerance: 1e-05
MSE after iteration 1: 0.03105490926975315, difference between errors: 0.022933922796415027, tolerance: 1e-05
MSE after iteration 2: 0.021863697606504813, difference between errors: 0.009191211663248336, tolerance: 1e-05
MSE after iteration 3: 0.01634580069002406, difference between errors: 0.005517896916480754, tolerance: 1e-05
MSE after iteration 4: 0.012438769375773493, difference between errors: 0.003907031314250567, tolerance: 1e-05
MSE after iteration 5: 0.00969157247306354, difference between errors: 0.0027471969027099523, tolerance: 1e-05
MSE after iteration 6: 0.007632561692907074, difference between errors: 0.0020590107801564663, tolerance: 1e-05
MSE after iteration 7: 0.0060669851337431056, difference between errors: 0.0015655765591639685, tolerance: 1e-05
MSE after iteration 8: 0.004934756725055955, difference between errors: 0.001132

In [20]:
matlab_eng.plot_file(os.path.join(data_dir, 'pcd_dummy.mat'), nargout=0)

In [21]:
# Clean the files
pcd_files = [x for x in get_files(
    transformed_data_dir, '.txt') if 'normal' not in x and 'clean' not in x]
normal_files = [x for x in get_files(
    transformed_data_dir, '.txt') if 'normal' in x and 'clean' not in x]
pcd_files, normal_files = clean_pcds(pcd_files[:], normal_files)

KeyboardInterrupt: 

In [22]:
# Specify the parameters
weight_types = [None, 'dist', 'dist5', 'dist55']
sampling_types = [None, 'iterative', 'uniform']
outlier_rej_types = [None, 'threshold', 'variance', 'normals']
steps = [1, 2, 4, 10]
weight_type = weight_types[0]
sampling_type = sampling_types[0]
outlier_rej_type = outlier_rej_types[0]
step = steps[0]

In [ ]:
# Run consecutive merging for all frames
start = time.time()

transformations,mean_error,times,mean_errors = get_merge_transformations(pcd_files, normal_files=normal_files, 
                                                                         weight_type=weight_type, step=step, method='consecutive', 
                                                                         outlier_rejection=outlier_rej_type, sample_size=5000, 
                                                                         sampling=sampling_type, tolerance=1e-10)

final_pcd = reduce(lambda src, target: merge_frames(src, target[0], target[1]),
                    transformations[1:],
                    transformations[0])

savemat(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_Consecutive.mat'),
        {'points': final_pcd})

print(f'Time elapsed: {time.time() - start}')

In [ ]:
matlab_eng.plot_file(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_Consecutive.mat'),
                     nargout=0)

In [ ]:
# Run consecutive merging for frames 1 to 50 and frames 100 to 50 and merge them
start = time.time()

half_pcd_files = pcd_files[:51]
second_half_pcd_files = pcd_files[50:][::-1]
half, _ = get_merge_transformations(half_pcd_files, normal_files=half_normal_files, weight_type=weight_type,
                                 step=step, method='consecutive', outlier_rejection=outlier_rej_type,
                                 sample_size=5000, sampling=sampling_type, tolerance=1e-11)
second_half, _ = get_merge_transformations(second_half_pcd_files, normal_files=second_half_normal_files, weight_type=weight_type,
                                        step=step, method='consecutive', outlier_rejection=outlier_rej_type,
                                        sample_size=5000, sampling=sampling_type, tolerance=1e-11)

final_pcd = reduce(lambda src, target: merge_frames(src, target[0], target[1]),
                    transformations[1:],
                    transformations[0])

final_pcd1 = reduce(lambda src, target: merge_frames(src, target[0], target[1]),
                    half[1:],
                    half[0])

final_pcd2 = reduce(lambda src, target: merge_frames(src, target[0], target[1]),
                    second_half[1:],
                    second_half[0])

final_pcd = np.concatenate((final_pcd1, final_pcd2), axis = 0)

savemat(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_half.mat'),
        {'points': final_pcd})

print(f'Time elapsed: {time.time() - start}')

In [ ]:
matlab_eng.plot_file(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_half.mat'),
                     nargout=0)

In [ ]:
# Run consecutive merging for the first 20 frames
start = time.time()

transformations,mean_error,times,mean_errors = get_merge_transformations(pcd_files[5:20], normal_files=normal_files[5:20], 
                                                                         weight_type=weight_type, step=step, method='iterative', 
                                                                         outlier_rejection=outlier_rej_type, sample_size=5000, 
                                                                         sampling=sampling_type, tolerance=1e-10)

final_pcd = reduce(lambda src, target: merge_frames(src, target[0], target[1]),
                    transformations[1:],
                    transformations[0])

savemat(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_Iterative.mat'),
        {'points': final_pcd})

print(f'Time elapsed: {time.time() - start}')

In [ ]:
matlab_eng.plot_file(os.path.join(data_dir, f'full_{sampling_type}_{weight_type}_{outlier_rej_type}_{step}_Iterative.mat'),
                     nargout=0)